In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topo_meas_queries, process_network, linknet, update_topology

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125


os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
[ConnNodeDict, EquipDict] = process_network.build_equip_dicts(gapps, model_mrid)

Processed  22 ConnectivyNode objects in  0.0002541099999999741 seconds
Processed  183 Measurement objects in  0.0003913629999999779 seconds
Processed  11 ACLineSegment objects in  0.0031926569999999876 seconds
Processed  14 Transformer objects in  0.005793431999999987 seconds
Processed  5 Switch objects in  0.008275089000000013 seconds


In [3]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
[ConnNodeDict, TerminalsDict, NodeList, TermList] = linknet.build_linknet_lists(ConnNodeDict, EquipDict, EqTypes)

Processed  11 ACLineSegment objects in  4.889000000002364e-05 seconds
Processed  2 PowerTransformer objects in  1.5344000000028224e-05 seconds
Processed  4 TransformerTank objects in  2.7242000000038402e-05 seconds
Processed  0 SynchronousMachine objects in  1.212000000000435e-05 seconds
Processed  1 missing nodes in  2.400899999999151e-05 seconds


In [4]:
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

In [ ]:
import time,json



SwitchKeys = list(EquipDict['Breaker'].keys()) + list(EquipDict['Fuse'].keys()) + list(EquipDict['LoadBreakSwitch'].keys()) + list(EquipDict['Recloser'].keys())
SwitchDict = {}
SwitchDict.update(EquipDict['Breaker'])
SwitchDict.update(EquipDict['Fuse'])
SwitchDict.update(EquipDict['LoadBreakSwitch'])
SwitchDict.update(EquipDict['Recloser'])

ConnNodeDict = json.loads(BaseConnDict)
TerminalsDict = json.loads(BaseTermDict)

StartTime = time.process_time()

for i5 in range(len(SwitchKeys)):

    node1=SwitchDict[SwitchKeys[i5]]['node1']
    node2=SwitchDict[SwitchKeys[i5]]['node2']


    # If switch closed, merge nodes
    if SwitchDict[SwitchKeys[i5]]['open'] == 1:
        # Merge topology Nodes
        #ConnNodeDict[node1]['TopologicalNode'] = tpnode1
        ConnNodeDict[node2]['TopologicalNode'] = ConnNodeDict[node1]['TopologicalNode'] #tpnode1
        #TopoNodeDict[tpnode1] = [node1, node2] # not implemented
        #TopoNodeDict[tpnode2] = [node2, node1]

        # Update Linked Lists
        if ConnNodeDict[node2]['list'] > ConnNodeDict[node1]['list']:
            term2 = TermList[ConnNodeDict[node2]['list']-1]
            next2 = TerminalsDict[term2]['next']
            while next2 != 0:
                term2 = TermList[next2-1]
                next2 = TerminalsDict[term2]['next']
            TerminalsDict[term2]['next'] = ConnNodeDict[node1]['list']
            ConnNodeDict[node1]['list'] = ConnNodeDict[node2]['list']
        else:
            term1 = TermList[ConnNodeDict[node1]['list']-1]
            next1 = TerminalsDict[term1]['next']
            while next1 != 0:
                term1 = TermList[next1-1]
                next1 = TerminalsDict[term1]['next']
            TerminalsDict[term1]['next'] = ConnNodeDict[node2]['list']
            ConnNodeDict[node2]['list'] = ConnNodeDict[node1]['list']

print("Processed ", i5+1, "switch objects in ", time.process_time() - StartTime, "seconds")



In [ ]:
[TerminalsDict, ConnNodeDict]=update_topology.topology_update(BaseConnDict,BaseTermDict,NodeList, TermList, EquipDict)

In [ ]:
SubstationRoots = []
XfmrKeys = list(EquipDict['PowerTransformer'])
# Iterate through all substation transformers
for i6 in range(len(XfmrKeys)):
    # Identify if distribution substation transformer
    if ((int(EquipDict['PowerTransformer'][XfmrKeys[i6]]['volt1']) >= 34000 and 34000 >= int(EquipDict['PowerTransformer'][XfmrKeys[i6]]['volt2']) > 1000) or 
        (int(EquipDict['PowerTransformer'][XfmrKeys[i6]]['volt2']) >= 34000 and 34000 >= int(EquipDict['PowerTransformer'][XfmrKeys[i6]]['volt1']) > 1000)):
        #SubstationRoots.append(EquipDict['PowerTransformer'][XfmrKeys[i6]]['node2'])
        SubstationRoots.append(XfmrKeys[i6])
        

In [ ]:
[SubTree, ConnNodeDict] = spanning_tree(ConnNodeDict, TerminalsDict, NodeList,  SubstationRoots, 'single')

In [ ]:
#def spanning_tree(ConnNodeDict, TerminalsDict, NodeList, RootNodes, Scope):
Scope = 'all'
RootNodes = SubstationRoots
eqtype = 'PowerTransformer'
    # Roots is list of ConnectivityNodes to be used as root nodes
# Terminals
Tree={}
#ProcessedNodes=[]
TotalNodes=0
StartTime = time.process_time()

# Iterate through all substation transformers
for i6 in range(len(RootNodes)):

    FirstNode = 1 
    LastNode = 2
    #Tree[RootNodes[i6]] = [RootNodes[i6]]
    # Set switch as rootnode 
    #if 'node2' not in EquipDict[eqtype][RootNodes[i6]]: # Check if 1-terminal object
    #    Tree[RootNodes[i6]]=[EquipDict[eqtype][RootNodes[i6]]['node1']]
    #else: # Then 2-terminal object
    Tree[RootNodes[i6]]=[EquipDict[eqtype][RootNodes[i6]]['node1'], EquipDict[eqtype][RootNodes[i6]]['node2']]

    while LastNode != FirstNode:
        NextTerm = ConnNodeDict[Tree[RootNodes[i6]][FirstNode]]['list']
        FirstNode = FirstNode + 1
        while NextTerm != 0:
            # Get next node and terminal for current node
            NextNode = TerminalsDict[TermList[NextTerm-1]]['far']
            NextTerm = TerminalsDict[TermList[NextTerm-1]]['next']
            node = NodeList[NextNode-1]
            not_in_tree = True

            if Scope == 'all': 
                for i7 in range(len(Tree.keys())):
                    if node in Tree[RootNodes[i7]]:
                        not_in_tree = False
                        break
            else: 
                if NodeList[NextNode-1] in Tree[RootNodes[i6]]: 
                    not_in_tree = False# Check if node already in all other tree


            # Add node if not in another tree        
            if not_in_tree:       
                if ConnNodeDict[node]['nominalVoltage']:
                    if int(ConnNodeDict[node]['nominalVoltage']) < 34000: #and int(ConnNodeDict[node]['nominalVoltage']) > 1000:
                        Tree[RootNodes[i6]].append(NodeList[NextNode-1])
                        LastNode = LastNode + 1                       
                else:
                    Tree[RootNodes[i6]].append(NodeList[NextNode-1])
                    LastNode = LastNode + 1

    NodesInTree=len(Tree[RootNodes[i6]])


    print("Processed topology from  ", RootNodes[i6], ' with ', NodesInTree, " buses")

    #print("Processed topology from  ", EquipDict[RootNodes[i6]]['name'], " with id ", RootNodes[i6], ' with ', NodesInTree, " buses")
    TotalNodes=TotalNodes+NodesInTree

print("Processed ", len(Tree.keys()), "topology trees containing ", TotalNodes, " buses in ", time.process_time() - StartTime, "seconds")

    #return Tree, ConnNodeDict

In [ ]:
Tree['_95DBCED6-E99E-4BB1-9229-899FB890050C']

In [ ]:
for i1 in range(1000):
    print(ConnNodeDict[Tree['_95DBCED6-E99E-4BB1-9229-899FB890050C'][i1]]['name'])

In [ ]:
TerminalsDict[TermList[4587]]

In [ ]:
TerminalsDict[TermList[2082]]

In [ ]:
NodeList[NextNode-1]


In [ ]:
ConnNodeDict['_E118EFA9-CF00-4DC5-9D6C-5D7486C0F7E5']

In [ ]:
ConnNodeDict['_0215D314-3A95-41D5-9071-DB8B7815850F']

In [ ]:
[ConnNodeDict[node2]['list']-1]

In [ ]:
ConnNodeDict[node2]

In [ ]:
NodeQuery[0]

In [ ]:
AllNodes = list(ConnNodeDict.keys())
MissingNodes = list(set(AllNodes).difference(NodeList))

In [ ]:
ConnNodeDict[MissingNodes[0]]

In [ ]:
ConnNodeDict[TerminalsDict[TermList[10432]]['ConnectivityNode']]

In [ ]:
import spanning_tree_old
import topologyqueries
XfmrDict = {}
XfmrDict.update(EquipDict['PowerTransformer'])
XfmrDict.update(EquipDict['TransformerTank'])
DG_query=topologyqueries.DGQuery(gapps, model_mrid)
[Tree,TotalNodes]=spanning_tree_old.generate_spanning_tree(XfmrKeys,XfmrDict,ConnNodeDict,TerminalsDict,TermList,NodeList,DG_query)


In [ ]:
EquipDict['RatioTapChanger']

In [ ]:
EquipDict['TransformerTank']

In [ ]:
SwitchDict